In [1]:
!pip install pandas==1.3.4
!pip install transformers==4.12.5
!pip install datasets==1.15.1

Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com


In [2]:
!pip install ipywidgets
!pip install IProgress

Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com


In [3]:
import os
import pickle

import pandas as pd

import numpy as np
import torch

import transformers
from transformers import BertTokenizer

import datasets
from datasets import concatenate_datasets
from datasets import Dataset
from datasets import ClassLabel
from datasets import DatasetDict

In [4]:
print('pandas:\t\t', pd.__version__)
print('transformers:\t', transformers.__version__)
print('datasets:\t', datasets.__version__)

pandas:		 1.3.4
transformers:	 4.12.5
datasets:	 1.15.1


## Load Data

In [5]:
dataset_df = pd.read_pickle("/notebooks/KURI-BERT/data/pe_dataset_w_essay_position_pos_tags_pickle")

In [6]:
dataset_df

,essay_nr,component_id,label_and_comp_idxs,text,label_x,label_ComponentType,relation_SupportAttack,label_RelationType,label_LinkedNotLinked,split_y,...,is_last_in_para,nr_preceeding_comps_in_para,nr_following_comps_in_para,structural_fts_as_text,structural_fts_as_text_combined,para_ratio,first_or_last,strct_fts_w_position_in_essay,component_pos_tags,strct_fts_essay_position_pos_tags
0,essay001,T1,MajorClaim 503 575,we should attach more importance to cooperatio...,MajorClaim,MajorClaim,[],,Linked,TRAIN,...,1,0,0,Topic: Should students be taught to compete or...,Topic: Should students be taught to compete or...,0.25,1,Topic: Should students be taught to compete or...,"Part Of Speech tags: PRON, VERB, VERB, ADJ, NO...",Topic: Should students be taught to compete or...
1,essay001,T2,MajorClaim 2154 2231,a more cooperative attitudes towards life is m...,MajorClaim,MajorClaim,[],,Linked,TRAIN,...,1,0,0,Topic: Should students be taught to compete or...,Topic: Should students be taught to compete or...,1.00,1,Topic: Should students be taught to compete or...,"Part Of Speech tags: DET, ADV, ADJ, NOUN, ADP,...",Topic: Should students be taught to compete or...
2,essay001,T3,Claim 591 714,"through cooperation, children can learn about ...",Claim,Claim,[],Support,Linked,TRAIN,...,0,0,3,Topic: Should students be taught to compete or...,Topic: Should students be taught to compete or...,0.50,0,Topic: Should students be taught to compete or...,"Part Of Speech tags: ADP, NOUN, PUNCT, NOUN, V...",Topic: Should students be taught to compete or...
3,essay001,T4,Premise 716 851,What we acquired from team work is not only ho...,Premise,Premise,[],Support,NotLinked,TRAIN,...,0,1,2,Topic: Should students be taught to compete or...,Topic: Should students be taught to compete or...,0.50,0,Topic: Should students be taught to compete or...,"Part Of Speech tags: PRON, PRON, VERB, ADP, NO...",Topic: Should students be taught to compete or...
4,essay001,T5,Premise 853 1086,"During the process of cooperation, children ca...",Premise,Premise,[],Support,NotLinked,TRAIN,...,0,2,1,Topic: Should students be taught to compete or...,Topic: Should students be taught to compete or...,0.50,0,Topic: Should students be taught to compete or...,"Part Of Speech tags: ADP, DET, NOUN, ADP, NOUN...",Topic: Should students be taught to compete or...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5968,essay402,T11,Premise 1275 1339,indirectly they will learn how to socialize ea...,Premise,Premise,[],Support,NotLinked,TRAIN,...,0,4,3,Topic: Children should studying hard or playin...,Topic: Children should studying hard or playin...,0.75,0,Topic: Children should studying hard or playin...,"Part Of Speech tags: ADV, PRON, VERB, VERB, AD...",Topic: Children should studying hard or playin...
5969,essay402,T12,Premise 1341 1388,That will make children getting lots of friends,Premise,Premise,[],Support,NotLinked,TRAIN,...,0,5,2,Topic: Children should studying hard or playin...,Topic: Children should studying hard or playin...,0.75,0,Topic: Children should studying hard or playin...,"Part Of Speech tags: DET, VERB, VERB, NOUN, VE...",Topic: Children should studying hard or playin...
5970,essay402,T13,Premise 1393 1436,they can contribute positively to community,Premise,Premise,[],Support,Linked,TRAIN,...,0,6,1,Topic: Children should studying hard or playin...,Topic: Children should studying hard or playin...,0.75,0,Topic: Children should studying hard or playin...,"Part Of Speech tags: PRON, VERB, VERB, ADV, AD...",Topic: Children should studying hard or playin...
5971,essay402,T14,Premise 1448 1525,playing sport makes children getting healthy a...,Premise,Premise,[],Support,NotLinked,TRAIN,...,1,7,0,Topic: Children should studying hard or playin...,Topic: Children should studying hard or playin...,0.75,0,Topic: Children should studying hard or playin...,"Part Of Speech tags: VERB, NOUN, VERB, NOUN, V...",Topic: Children should studying hard or

#### get ams

In [7]:
import re
sentence = "Consequently, no matter from the view of individual development or the relationship between competition and cooperation we can receive the same conclusion that a more cooperative attitudes towards life is more profitable in one's success." 
x = re.findall(r'\s|,|[^,\s]+', sentence)

In [8]:
x

['Consequently',
 ',',
 ' ',
 'no',
 ' ',
 'matter',
 ' ',
 'from',
 ' ',
 'the',
 ' ',
 'view',
 ' ',
 'of',
 ' ',
 'individual',
 ' ',
 'development',
 ' ',
 'or',
 ' ',
 'the',
 ' ',
 'relationship',
 ' ',
 'between',
 ' ',
 'competition',
 ' ',
 'and',
 ' ',
 'cooperation',
 ' ',
 'we',
 ' ',
 'can',
 ' ',
 'receive',
 ' ',
 'the',
 ' ',
 'same',
 ' ',
 'conclusion',
 ' ',
 'that',
 ' ',
 'a',
 ' ',
 'more',
 ' ',
 'cooperative',
 ' ',
 'attitudes',
 ' ',
 'towards',
 ' ',
 'life',
 ' ',
 'is',
 ' ',
 'more',
 ' ',
 'profitable',
 ' ',
 'in',
 ' ',
 "one's",
 ' ',
 'success.']

In [9]:
clean_x = [item for item in x if item != " "]

In [10]:
clean_x

['Consequently',
 ',',
 'no',
 'matter',
 'from',
 'the',
 'view',
 'of',
 'individual',
 'development',
 'or',
 'the',
 'relationship',
 'between',
 'competition',
 'and',
 'cooperation',
 'we',
 'can',
 'receive',
 'the',
 'same',
 'conclusion',
 'that',
 'a',
 'more',
 'cooperative',
 'attitudes',
 'towards',
 'life',
 'is',
 'more',
 'profitable',
 'in',
 "one's",
 'success.']

In [11]:
with open("am_list.txt") as file_am:
    am_list = []
    for line in file_am:
        am_list.append(line.rstrip('\n'))

In [12]:
am_list

['Additionally',
 'As a matter of fact',
 'because',
 'Besides',
 'due to',
 'Finally',
 'First of all',
 'Firstly',
 'For example',
 'for instance',
 'Furthermore,',
 'has proved it',
 'In addition',
 'In addition to this',
 'In the first place',
 'is due to the fact that',
 'It should also be noted',
 'Moreover',
 'On one hand',
 'On the one hand',
 'On the other hand',
 'One of the main reasons',
 'Secondly',
 'Similarly',
 'since',
 'So',
 'The reason',
 'To begin with',
 'To offer an instance',
 'What is more',
 'As a result',
 'As the consequence',
 'Clearly',
 'Consequently',
 'Considering this subject',
 'Furthermore',
 'Hence',
 'leading to the consequence',
 'so',
 'taking account on this fact',
 'That is the reason why',
 'The reason is that',
 'Therefore,',
 'This means that',
 'This shows that',
 'This will result',
 'Thus',
 'Thus, it is clearly seen that',
 'Thus, it is seen',
 'Thus, the example shows',
 'Admittedly',
 'although ',
 'besides these advantages',
 'but',
 

In [13]:
'First of all'.split(" ")

['First', 'of', 'all']

In [14]:
def am_in_prefix(list_a, list_b):
    
    equal_flag = 0
    
    len_list_b = len(list_b)
    
    for i in range(len(list_a)):
        
        sub_list = list_a[i:i+len_list_b]
        
        if sub_list == list_b:
            
            equal_flag = 1
            break
        
        else:
            
            equal_flag = 0
            
    return equal_flag

In [15]:
def get_am(x, am_list):
    
    am = []
    
    sentence = x.sentence
    component = x.text
    
    prefix = sentence[:sentence.find(component)]
    prefix = re.findall(r'\s|,|[^,\s]+', prefix) 
    prefix = [item for item in prefix if item != " "]
    prefix = [word.casefold() for word in prefix]
    
    am_list = [am.casefold() for am in am_list]
    am_list = list(set(am_list))
    
    # am_list = [am.]
    
    for am_phrase in am_list:
        
        am_words_list = re.findall(r'\s|,|[^,\s]+', am_phrase)
        am_words_list = [item for item in am_words_list if item != " "]
        
        if am_in_prefix(prefix, am_words_list) == 1 and len(am) == 0:
            
            am.append(am_phrase)
    
    return am         

In [16]:
get_am(dataset_df.iloc[1], am_list)

['consequently']

In [17]:
dataset_df['am'] = dataset_df.apply(lambda x: get_am(x, am_list), axis=1)

In [18]:
dataset_df['am']

0       [from this point of view]
1                  [consequently]
2                  [first of all]
3                              []
4                              []
                  ...            
5968                    [because]
5969                           []
5970                         [so]
5971                   [secondly]
5972                           []
Name: am, Length: 5973, dtype: object

In [19]:
dataset_df['sentence'][5972], dataset_df['text'][5972]

('At the same time, playing sports will give good effects on children',
 'playing sports will give good effects on children')

In [20]:
# note: maybe the longest one instead of teh first one.

### do group by for every required column

In [21]:
paras_df = dataset_df.groupby("paragraph", sort=False)

In [22]:
paras_df

In [23]:
df_by_paras_text = paras_df["text"].apply(list)

In [24]:
df_by_paras_text

paragraph
It is always said that competition can effectively promote the development of economy. In order to survive in the competition, companies continue to improve their products and service, and as a result, the whole society prospers. However, when we discuss the issue of competition or cooperation, what we are concerned about is not the whole society, but the development of an individual's whole life. From this point of view, I firmly believe that we should attach more importance to cooperation during primary education.                                                                                                                                                                                                                                                                                           [we should attach more importance to cooperati...
Consequently, no matter from the view of individual development or the relationship between competition and cooperation we can receive t

In [25]:
df_by_paras_text = df_by_paras_text.reset_index()

In [26]:
df_by_paras_text

,paragraph,text
0,It is always said that competition can effecti...,[we should attach more importance to cooperati...
1,"Consequently, no matter from the view of indiv...",[a more cooperative attitudes towards life is ...
2,"First of all, through cooperation, children ca...","[through cooperation, children can learn about..."
3,"On the other hand, the significance of competi...","[competition makes the society more effective,..."
4,The last 50 years have seen an increasing numb...,[they are able to sustain their cultural ident...
...,...,...
1714,The second reason why I believe that fatherood...,[we are living in a society that puts a signif...
1715,Some people believe that studying hard is esse...,[both of studying hard and playing sports are ...
1716,"In conclusion, both studying hard at school an...",[both studying hard at school and playing spor...
1717,"On the other hand, studying hard will give chi...",[Studying hard gives children with a plenty of...


In [27]:
df_by_paras_labels = paras_df["label_ComponentType"].apply(list)

In [28]:
df_by_paras_labels

paragraph
It is always said that competition can effectively promote the development of economy. In order to survive in the competition, companies continue to improve their products and service, and as a result, the whole society prospers. However, when we discuss the issue of competition or cooperation, what we are concerned about is not the whole society, but the development of an individual's whole life. From this point of view, I firmly believe that we should attach more importance to cooperation during primary education.                                                                                                                                                                                                                                                                                                                                [MajorClaim]
Consequently, no matter from the view of individual development or the relationship between competition and cooperation we can receive t

In [29]:
df_by_paras_labels = df_by_paras_labels.reset_index()

In [30]:
df_by_paras_labels

,paragraph,label_ComponentType
0,It is always said that competition can effecti...,[MajorClaim]
1,"Consequently, no matter from the view of indiv...",[MajorClaim]
2,"First of all, through cooperation, children ca...","[Claim, Premise, Premise, Premise]"
3,"On the other hand, the significance of competi...","[Claim, Premise, Premise, Premise, Claim]"
4,The last 50 years have seen an increasing numb...,[MajorClaim]
...,...,...
1714,The second reason why I believe that fatherood...,"[Premise, Premise, Claim, Premise, Premise]"
1715,Some people believe that studying hard is esse...,[MajorClaim]
1716,"In conclusion, both studying hard at school an...",[MajorClaim]
1717,"On the other hand, studying hard will give chi...","[Premise, Premise, Premise, Premise, Claim]"


In [31]:
# Nice.

In [32]:
df_by_paras_ams = paras_df["am"].apply(list)

In [33]:
df_by_paras_ams

paragraph
It is always said that competition can effectively promote the development of economy. In order to survive in the competition, companies continue to improve their products and service, and as a result, the whole society prospers. However, when we discuss the issue of competition or cooperation, what we are concerned about is not the whole society, but the development of an individual's whole life. From this point of view, I firmly believe that we should attach more importance to cooperation during primary education.                                                                                                                                                                                                                                                                                                                 [[from this point of view]]
Consequently, no matter from the view of individual development or the relationship between competition and cooperation we can receive t

In [34]:
df_by_paras_ams = df_by_paras_ams.reset_index()

In [35]:
df_by_paras_ams

,paragraph,am
0,It is always said that competition can effecti...,[[from this point of view]]
1,"Consequently, no matter from the view of indiv...",[[consequently]]
2,"First of all, through cooperation, children ca...","[[first of all], [], [], []]"
3,"On the other hand, the significance of competi...","[[hence], [on the other hand], [however], [], []]"
4,The last 50 years have seen an increasing numb...,[[however]]
...,...,...
1714,The second reason why I believe that fatherood...,"[[in my opinion], [for instance], [], [], [the..."
1715,Some people believe that studying hard is esse...,[[in my point of view]]
1716,"In conclusion, both studying hard at school an...",[[in conclusion]]
1717,"On the other hand, studying hard will give chi...","[[], [for instance], [], [], [on the other hand]]"


In [36]:
# make the first two DFs lists of lists

In [37]:
def make_acs_l_o_l(x):
    
    list_acs = x.text
    
    new_list = [[ac] for ac in list_acs]
    
    return new_list

In [38]:
df_by_paras_text['paragraph_components_list'] = df_by_paras_text.apply(lambda x: make_acs_l_o_l(x), axis=1)

In [39]:
df_by_paras_text['paragraph_components_list'][3]

[['competition makes the society more effective'],
 ['the significance of competition is that how to become more excellence to gain the victory'],
 ['when we consider about the question that how to win the game, we always find that we need the cooperation'],
 ['Take Olympic games which is a form of competition for instance, it is hard to imagine how an athlete could win the game without the training of his or her coach, and the help of other professional staffs such as the people who take care of his diet, and those who are in charge of the medical care'],
 ['without the cooperation, there would be no victory of competition']]

In [40]:
# ok now for the second labels df

In [41]:
def make_labels_l_o_l(x):
    
    list_labels = x.label_ComponentType
    
    new_list = [[label] for label in list_labels]
    
    return new_list

In [42]:
df_by_paras_labels['paragraph_labels_list'] = df_by_paras_labels.apply(lambda x: make_labels_l_o_l(x), axis=1)

In [43]:
df_by_paras_labels['paragraph_labels_list'][3]

[['Claim'], ['Premise'], ['Premise'], ['Premise'], ['Claim']]

In [44]:
# ok seems correct.

In [45]:
# now combine the three dataframes

In [46]:
df = pd.concat([df_by_paras_text, df_by_paras_labels, df_by_paras_ams], axis=1)

In [47]:
df

,paragraph,text,paragraph_components_list,paragraph,label_ComponentType,paragraph_labels_list,paragraph,am
0,It is always said that competition can effecti...,[we should attach more importance to cooperati...,[[we should attach more importance to cooperat...,It is always said that competition can effecti...,[MajorClaim],[[MajorClaim]],It is always said that competition can effecti...,[[from this point of view]]
1,"Consequently, no matter from the view of indiv...",[a more cooperative attitudes towards life is ...,[[a more cooperative attitudes towards life is...,"Consequently, no matter from the view of indiv...",[MajorClaim],[[MajorClaim]],"Consequently, no matter from the view of indiv...",[[consequently]]
2,"First of all, through cooperation, children ca...","[through cooperation, children can learn about...","[[through cooperation, children can learn abou...","First of all, through cooperation, children ca...","[Claim, Premise, Premise, Premise]","[[Claim], [Premise], [Premise], [Premise]]","First of all, through cooperation, children ca...","[[first of all], [], [], []]"
3,"On the other hand, the significance of competi...","[competition makes the society more effective,...",[[competition makes the society more effective...,"On the other hand, the significance of competi...","[Claim, Premise, Premise, Premise, Claim]","[[Claim], [Premise], [Premise], [Premise], [Cl...","On the other hand, the significance of competi...","[[hence], [on the other hand], [however], [], []]"
4,The last 50 years have seen an increasing numb...,[they are able to sustain their cultural ident...,[[they are able to sustain their cultural iden...,The last 50 years have seen an increasing numb...,[MajorClaim],[[MajorClaim]],The last 50 years have seen an increasing numb...,[[however]]
...,...,...,...,...,...,...,...,...
1714,The second reason why I believe that fatherood...,[we are living in a society that puts a signif...,[[we are living in a society that puts a signi...,The second reason why I believe that fatherood...,"[Premise, Premise, Claim, Premise, Premise]","[[Premise], [Premise], [Claim], [Premise], [Pr...",The second reason why I believe that fatherood...,"[[in my opinion], [for instance], [], [], [the..."
1715,Some people believe that studying hard is esse...,[both of studying hard and playing sports are ...,[[both of studying hard and playing sports are...,Some people believe that studying hard is esse...,[MajorClaim],[[MajorClaim]],Some people believe that studying hard is esse...,[[in my point of view]]
1716,"In conclusion, both studying hard at school an...",[both studying hard at school and playing spor...,[[both studying hard at school and playing spo...,"In conclusion, both studying hard at school an...",[MajorClaim],[[MajorClaim]],"In conclusion, both studying hard at school an...",[[in conclusion]]
1717,"On the other hand, studying hard will give chi...",[Studying hard gives children with a plenty of...,[[Studying hard gives children with a plenty o...,"On the other hand, studying hard will give chi...","[Premise, Premise, Premise, Premise, Claim]","[[Premise], [Premise], [Premise], [Premise], [...","On the other hand, studying hard will give chi...","[[], [for instance], [], [], [on the other hand]]"


In [48]:
df = df.loc[:,~df.columns.duplicated()].copy()

In [49]:
df

,paragraph,text,paragraph_components_list,label_ComponentType,paragraph_labels_list,am
0,It is always said that competition can effecti...,[we should attach more importance to cooperati...,[[we should attach more importance to cooperat...,[MajorClaim],[[MajorClaim]],[[from this point of view]]
1,"Consequently, no matter from the view of indiv...",[a more cooperative attitudes towards life is ...,[[a more cooperative attitudes towards life is...,[MajorClaim],[[MajorClaim]],[[consequently]]
2,"First of all, through cooperation, children ca...","[through cooperation, children can learn about...","[[through cooperation, children can learn abou...","[Claim, Premise, Premise, Premise]","[[Claim], [Premise], [Premise], [Premise]]","[[first of all], [], [], []]"
3,"On the other hand, the significance of competi...","[competition makes the society more effective,...",[[competition makes the society more effective...,"[Claim, Premise, Premise, Premise, Claim]","[[Claim], [Premise], [Premise], [Premise], [Cl...","[[hence], [on the other hand], [however], [], []]"
4,The last 50 years have seen an increasing numb...,[they are able to sustain their cultural ident...,[[they are able to sustain their cultural iden...,[MajorClaim],[[MajorClaim]],[[however]]
...,...,...,...,...,...,...
1714,The second reason why I believe that fatherood...,[we are living in a society that puts a signif...,[[we are living in a society that puts a signi...,"[Premise, Premise, Claim, Premise, Premise]","[[Premise], [Premise], [Claim], [Premise], [Pr...","[[in my opinion], [for instance], [], [], [the..."
1715,Some people believe that studying hard is esse...,[both of studying hard and playing sports are ...,[[both of studying hard and playing sports are...,[MajorClaim],[[MajorClaim]],[[in my point of view]]
1716,"In conclusion, both studying hard at school an...",[both studying hard at school and playing spor...,[[both studying hard at school and playing spo...,[MajorClaim],[[MajorClaim]],[[in conclusion]]
1717,"On the other hand, studying hard will give chi...",[Studying hard gives children with a plenty of...,[[Studying hard gives children with a plenty o...,"[Premise, Premise, Premise, Premise, Claim]","[[Premise], [Premise], [Premise], [Premise], [...","[[], [for instance], [], [], [on the other hand]]"


In [50]:
df = df.drop(columns=['text', 'label_ComponentType'])

In [51]:
df = df.rename(columns={"am": "paragraph_markers_list"})

In [52]:
df

,paragraph,paragraph_components_list,paragraph_labels_list,paragraph_markers_list
0,It is always said that competition can effecti...,[[we should attach more importance to cooperat...,[[MajorClaim]],[[from this point of view]]
1,"Consequently, no matter from the view of indiv...",[[a more cooperative attitudes towards life is...,[[MajorClaim]],[[consequently]]
2,"First of all, through cooperation, children ca...","[[through cooperation, children can learn abou...","[[Claim], [Premise], [Premise], [Premise]]","[[first of all], [], [], []]"
3,"On the other hand, the significance of competi...",[[competition makes the society more effective...,"[[Claim], [Premise], [Premise], [Premise], [Cl...","[[hence], [on the other hand], [however], [], []]"
4,The last 50 years have seen an increasing numb...,[[they are able to sustain their cultural iden...,[[MajorClaim]],[[however]]
...,...,...,...,...
1714,The second reason why I believe that fatherood...,[[we are living in a society that puts a signi...,"[[Premise], [Premise], [Claim], [Premise], [Pr...","[[in my opinion], [for instance], [], [], [the..."
1715,Some people believe that studying hard is esse...,[[both of studying hard and playing sports are...,[[MajorClaim]],[[in my point of view]]
1716,"In conclusion, both studying hard at school an...",[[both studying hard at school and playing spo...,[[MajorClaim]],[[in conclusion]]
1717,"On the other hand, studying hard will give chi...",[[Studying hard gives children with a plenty o...,"[[Premise], [Premise], [Premise], [Premise], [...","[[], [for instance], [], [], [on the other hand]]"


In [53]:
def is_equal(x):
    
    flag = 0
    
    list_ac = x.paragraph_components_list
    list_am = x.paragraph_markers_list
    
    if len(list_ac) == len(list_am):
        
        flag = 1
        
    else:
        
        flag = 0
        
    return flag

In [54]:
df['ac=am'] = df.apply(lambda x: is_equal(x), axis=1)

In [55]:
df['ac=am'].value_counts()

1    1719
Name: ac=am, dtype: int64

In [56]:
def is_equal_label(x):
    
    flag = 0
    
    list_ac = x.paragraph_components_list
    list_am = x.paragraph_labels_list
    
    if len(list_ac) == len(list_am):
        
        flag = 1
        
    else:
        
        flag = 0
        
    return flag

In [57]:
df['ac=lab'] = df.apply(lambda x: is_equal_label(x), axis=1)

In [58]:
df['ac=lab'].value_counts()

1    1719
Name: ac=lab, dtype: int64

In [59]:
# ok both correct.

In [60]:
df = df.drop(columns=df.columns[-1])

In [61]:
df

,paragraph,paragraph_components_list,paragraph_labels_list,paragraph_markers_list,ac=am
0,It is always said that competition can effecti...,[[we should attach more importance to cooperat...,[[MajorClaim]],[[from this point of view]],1
1,"Consequently, no matter from the view of indiv...",[[a more cooperative attitudes towards life is...,[[MajorClaim]],[[consequently]],1
2,"First of all, through cooperation, children ca...","[[through cooperation, children can learn abou...","[[Claim], [Premise], [Premise], [Premise]]","[[first of all], [], [], []]",1
3,"On the other hand, the significance of competi...",[[competition makes the society more effective...,"[[Claim], [Premise], [Premise], [Premise], [Cl...","[[hence], [on the other hand], [however], [], []]",1
4,The last 50 years have seen an increasing numb...,[[they are able to sustain their cultural iden...,[[MajorClaim]],[[however]],1
...,...,...,...,...,...
1714,The second reason why I believe that fatherood...,[[we are living in a society that puts a signi...,"[[Premise], [Premise], [Claim], [Premise], [Pr...","[[in my opinion], [for instance], [], [], [the...",1
1715,Some people believe that studying hard is esse...,[[both of studying hard and playing sports are...,[[MajorClaim]],[[in my point of view]],1
1716,"In conclusion, both studying hard at school an...",[[both studying hard at school and playing spo...,[[MajorClaim]],[[in conclusion]],1
1717,"On the other hand, studying hard will give chi...",[[Studying hard gives children with a plenty o...,"[[Premise], [Premise], [Premise], [Premise], [...","[[], [for instance], [], [], [on the other hand]]",1


In [62]:
def get_numeric_labels(x):
    
    numeric_labels = []
    
    string_labels = x.paragraph_labels_list
    
    for label in string_labels:
        
        if label == ['MajorClaim']:
            
            numeric_labels.append(0)
            
        elif label == ['Claim']:
            
            numeric_labels.append(1)
            
        elif label == ['Premise']:
            
            numeric_labels.append(2)
            
    return numeric_labels

In [63]:
df['paragraph_labels_numeric'] = df.apply(lambda x: get_numeric_labels(x), axis=1)

In [64]:
lab_list = df['paragraph_labels_numeric'].tolist()

In [65]:
lab_list

[[0],
 [0],
 [1, 2, 2, 2],
 [1, 2, 2, 2, 1],
 [0],
 [0, 1],
 [2, 1, 2, 2, 2],
 [2, 2, 1, 2, 2, 2],
 [0],
 [0],
 [1, 2, 2, 2],
 [2, 1, 2, 2],
 [0, 1],
 [0],
 [1, 2, 2, 2],
 [1, 2, 2, 2],
 [0],
 [0, 1],
 [1, 2],
 [1, 2, 2, 2],
 [1, 2, 2],
 [0, 1],
 [0],
 [1, 2, 2, 2, 2],
 [1, 2, 2],
 [1, 2, 2, 2, 2, 2, 2, 2],
 [0],
 [1, 2, 2, 2],
 [1, 2, 2, 2],
 [1, 2, 2],
 [0],
 [0, 1],
 [1, 2, 2, 2],
 [1, 2, 2, 2, 2, 2],
 [0],
 [0, 0],
 [2, 2, 2, 2, 1, 2],
 [2, 2, 1, 2, 2, 2, 2],
 [0, 1, 2],
 [1, 2, 2, 2, 2, 1],
 [2, 2, 1, 2, 2],
 [1, 0, 1],
 [0, 1],
 [0],
 [1, 2, 2, 2, 2],
 [1, 2, 2, 2],
 [1, 2, 2, 2, 2, 1, 2],
 [1, 2, 2],
 [2, 2, 1, 2],
 [0, 1],
 [2, 2, 2, 2, 1],
 [0, 1],
 [1, 2, 2, 0, 2],
 [2, 2, 2, 2, 2, 2, 1],
 [2, 2, 2, 2, 2, 1],
 [1, 2, 2, 2, 2],
 [0, 1],
 [2, 2, 2, 2, 2, 1, 2],
 [0],
 [1, 2, 2, 2, 2, 2, 2],
 [0],
 [1, 2, 2, 2, 2, 2],
 [1, 0],
 [0],
 [1, 2, 2, 2],
 [2, 2, 1, 2, 2, 2],
 [0, 1],
 [0, 1],
 [1, 2, 2, 2, 2],
 [1, 2, 2, 2],
 [1, 2, 2, 2],
 [0, 1, 2, 2, 2],
 [1, 2, 2, 2, 2, 2],
 [1, 2,

In [66]:
lab_list

[[0],
 [0],
 [1, 2, 2, 2],
 [1, 2, 2, 2, 1],
 [0],
 [0, 1],
 [2, 1, 2, 2, 2],
 [2, 2, 1, 2, 2, 2],
 [0],
 [0],
 [1, 2, 2, 2],
 [2, 1, 2, 2],
 [0, 1],
 [0],
 [1, 2, 2, 2],
 [1, 2, 2, 2],
 [0],
 [0, 1],
 [1, 2],
 [1, 2, 2, 2],
 [1, 2, 2],
 [0, 1],
 [0],
 [1, 2, 2, 2, 2],
 [1, 2, 2],
 [1, 2, 2, 2, 2, 2, 2, 2],
 [0],
 [1, 2, 2, 2],
 [1, 2, 2, 2],
 [1, 2, 2],
 [0],
 [0, 1],
 [1, 2, 2, 2],
 [1, 2, 2, 2, 2, 2],
 [0],
 [0, 0],
 [2, 2, 2, 2, 1, 2],
 [2, 2, 1, 2, 2, 2, 2],
 [0, 1, 2],
 [1, 2, 2, 2, 2, 1],
 [2, 2, 1, 2, 2],
 [1, 0, 1],
 [0, 1],
 [0],
 [1, 2, 2, 2, 2],
 [1, 2, 2, 2],
 [1, 2, 2, 2, 2, 1, 2],
 [1, 2, 2],
 [2, 2, 1, 2],
 [0, 1],
 [2, 2, 2, 2, 1],
 [0, 1],
 [1, 2, 2, 0, 2],
 [2, 2, 2, 2, 2, 2, 1],
 [2, 2, 2, 2, 2, 1],
 [1, 2, 2, 2, 2],
 [0, 1],
 [2, 2, 2, 2, 2, 1, 2],
 [0],
 [1, 2, 2, 2, 2, 2, 2],
 [0],
 [1, 2, 2, 2, 2, 2],
 [1, 0],
 [0],
 [1, 2, 2, 2],
 [2, 2, 1, 2, 2, 2],
 [0, 1],
 [0, 1],
 [1, 2, 2, 2, 2],
 [1, 2, 2, 2],
 [1, 2, 2, 2],
 [0, 1, 2, 2, 2],
 [1, 2, 2, 2, 2, 2],
 [1, 2,

In [67]:
def label_padding(x):
    
    list_labels = x.paragraph_labels_numeric
    
    pads_needed = 12 - len(list_labels)
    
    for i in range(pads_needed):
        
        list_labels.append(-1)
        
    return list_labels

In [68]:
df['paragraph_labels_numeric_padded'] = df.apply(lambda x: label_padding(x), axis=1)

In [69]:
df['paragraph_labels_numeric_padded']

0       [0, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1]
1       [0, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1]
2          [1, 2, 2, 2, -1, -1, -1, -1, -1, -1, -1, -1]
3           [1, 2, 2, 2, 1, -1, -1, -1, -1, -1, -1, -1]
4       [0, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1]
                             ...                       
1714        [2, 2, 1, 2, 2, -1, -1, -1, -1, -1, -1, -1]
1715    [0, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1]
1716    [0, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1]
1717        [2, 2, 2, 2, 1, -1, -1, -1, -1, -1, -1, -1]
1718           [2, 2, 2, 2, 2, 2, 2, 1, -1, -1, -1, -1]
Name: paragraph_labels_numeric_padded, Length: 1719, dtype: object

In [71]:
# correct the five problematic ones

In [73]:
df['paragraph_components_list'][410] = [['when children take jobs, they tend to be more responsible'],
 ['whether they can earn money or not will depend on their effectiveness and attitudes in working'],
 ['To keep their jobs, children will have to try their best to finish their duties'],
 ['if a child works as a shop assistant, he will have to keep an eye on good and products, calculate prices correctly and take responsibilities for any lost']]

/tmp/ipykernel_357/2276292281.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['paragraph_components_list'][410] = [['when children take jobs, they tend to be more responsible'],


In [74]:
df['paragraph_components_list'][410]

[['when children take jobs, they tend to be more responsible'],
 ['whether they can earn money or not will depend on their effectiveness and attitudes in working'],
 ['To keep their jobs, children will have to try their best to finish their duties'],
 ['if a child works as a shop assistant, he will have to keep an eye on good and products, calculate prices correctly and take responsibilities for any lost']]

In [75]:
df['paragraph_components_list'][474] = [['children need human interactions'],
 ["As a recent interview with Josh Trumm – the founder of the 'Teachers Global Group'- reveals, human interactions will never be imitated by computers: 'This verbal and non-verbal communication between teachers and children is the first step towards a proper education"],
 ["Without someone who can explain and understand their problems, pupils cannot actually learn'"],
 ['only a teacher can feel and connect with pupils, be a friend and a model'],
 ['No matter how advanced technology will be, it is clear that a computer will never be able to communicate with its pupils']]

/tmp/ipykernel_357/3590879287.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['paragraph_components_list'][474] = [['children need human interactions'],


In [76]:
df['paragraph_components_list'][768] = [['the individual should finance his or her education'],
 ['it would cause too much cost from taxpayers and the government']]

/tmp/ipykernel_357/2797774357.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['paragraph_components_list'][768] = [['the individual should finance his or her education'],


In [77]:
df['paragraph_components_list'][1062] = [['as parents are involved in the same environment'],
 ['my father is a software programmer, and he indeed teaches me a lot in programming and some computer knowledge'],
 ['Whenever I met a difficulty, I would reach him for consulting'],
 ['he has many experiences'],
 ['I could save enormous time in finding solution'],
 ['This truly makes me more competitive than my colleagues'],
 ['If I worked in the different field from my father, I would have faced much difficulty than I do now'],
 ['they could thus support much more to their children, such as domain knowledge and industry-related information']]

/tmp/ipykernel_357/2408733210.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['paragraph_components_list'][1062] = [['as parents are involved in the same environment'],


In [78]:
df['paragraph_components_list'][1470] = [['as lots of chatting apps available online such as "Weichat", people tend to send instant messages free of charge by using their phones rather than face to face communication'],
 ['mobile phones have shortened the distance of communication'],
 ['With rapid development of the internet, people are able to enjoy quick electronic communication via internet'],
 ['mobile phones have shortened the distance of communication']]

/tmp/ipykernel_357/3257930866.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['paragraph_components_list'][1470] = [['as lots of chatting apps available online such as "Weichat", people tend to send instant messages free of charge by using their phones rather than face to face communication'],


#### dataset

In [79]:
dataset_d = {
    'paragraph' : df['paragraph'],
    'paragraph_ACs' : df['paragraph_components_list'],
    'paragraph_AMs' : df['paragraph_markers_list'],
    'paragraph_labels' : df['paragraph_labels_numeric_padded']
}

In [80]:
dataset = datasets.Dataset.from_dict(dataset_d)

In [81]:
dataset

Dataset({
    features: ['paragraph', 'paragraph_ACs', 'paragraph_AMs', 'paragraph_labels'],
    num_rows: 1719
})

In [82]:
dataset['paragraph_AMs']

[[['from this point of view']],
 [['consequently']],
 [['first of all'], [], [], []],
 [['hence'], ['on the other hand'], ['however'], [], []],
 [['however']],
 [['as far as i am concerned'], ['although ']],
 [['firstly'], ['thus'], [], [], []],
 [['secondly'], ['secondly'], ['hence'], ['for instance'], [], []],
 [[]],
 [['as far as i am concerned']],
 [[], ['firstly'], [], []],
 [['secondly'], ['thus'], ['for instance'], []],
 [[], []],
 [['to conclude']],
 [[], ['firstly'], [], []],
 [['thus'], ['secondly'], [], []],
 [['in my opinion']],
 [['however'], ['in conclusion']],
 [[], []],
 [[], [], ['but'], ['moreover']],
 [['therefore,'], [], []],
 [['i believe'], []],
 [['in conclusion']],
 [['first of all'], [], ['for example'], ['therefore,'], []],
 [[], ['for example'], []],
 [[], [], ['furthermore'], [], [], [], [], []],
 [['however']],
 [['hence'], ['first of all'], [], []],
 [[], [], [], []],
 [[], ['but'], []],
 [[]],
 [[], []],
 [[], ['so'], [], []],
 [[], [], [], [], [], ['but'

In [83]:
# now get the nice align function

In [84]:
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")

In [85]:
def tokenize_get_spans_align(dataset):
    
    tokenized_text = tokenizer(dataset['paragraph'], padding=True, return_tensors="pt")
    tokenized_text['labels'] = torch.tensor(dataset['paragraph_labels'])
    
    # now get the spans
    # what if am is empty.
    
    ac_spans = []
    am_spans = []
    
    for idx, sample in enumerate(dataset):
        
        # print(idx)
        

        
        paragraph = sample['paragraph']
        ac_list = sample['paragraph_ACs']
        am_list = sample['paragraph_AMs']
    
        paragraph_tokenized = tokenizer.tokenize(paragraph)
        
        # do ACs. ACs cannot be empty.
        
        
        for ac in ac_list:            
            
            ac_tokenized = tokenizer.tokenize(ac[0])
            ac_span = [(i, i+len(ac_tokenized)-1) for i in range(len(paragraph_tokenized)) if paragraph_tokenized[i:i+len(ac_tokenized)] == ac_tokenized]
            ac_spans.append(ac_span)
            
        for am in am_list:
            
            if len(am) == 0:
                
                am_span = [(-1,-1)] # it should just be a tuple. see about this later.
                
            else:
                
                am_tokenized = tokenizer.tokenize(am[0])
                am_span = [(i, i+len(am_tokenized)-1) for i in range(len(paragraph_tokenized)) if paragraph_tokenized[i:i+len(am_tokenized)] == am_tokenized]
            
            am_spans.append(am_span)
            
    print(len(am_spans))
    print(len(ac_spans))
                
    tokenized_text['spans'] = ac_spans # then correct this.
    
    return tokenized_text

In [86]:
tokenized_input = tokenize_get_spans_align(dataset)

5973
5973


In [87]:
# nice! so this is good. every ADU has an AM span and an AC span.
# the AM span can be empty. AC cannot.
# but now these need to be changed to a paragraph level

In [198]:
def tokenize_get_spans_align(dataset):
    
    tokenized_text = tokenizer(dataset['paragraph'], padding=True, return_tensors="pt")
    tokenized_text['labels'] = torch.tensor(dataset['paragraph_labels'])
    
    # now get the spans
    # what if am is empty.
    
    # ac_spans = []
    # am_spans = []
    
    spans = []
    
    for idx, sample in enumerate(dataset):
        
        para_spans = []
        sample_am_spans = []
        sample_ac_spans = []
        
        
        paragraph = sample['paragraph']
        ac_list = sample['paragraph_ACs']
        am_list = sample['paragraph_AMs']
    
        paragraph_tokenized = tokenizer.tokenize(paragraph)
        
        # do ACs. ACs cannot be empty.
        
        for ac in ac_list:            
            
            ac_tokenized = tokenizer.tokenize(ac[0])
            ac_span = [(i, i+len(ac_tokenized)-1) for i in range(len(paragraph_tokenized)) if paragraph_tokenized[i:i+len(ac_tokenized)] == ac_tokenized]
            if len(ac_span) > 1:
                    # print(am_span)
                    ac_span = [ac_span[0]]
            sample_ac_spans.append(ac_span)
            
        for am in am_list:
            
            if not am:
                
                am_span = [(-1,-1)]
                
            else:
                
                am_tokenized = tokenizer.tokenize(am[0])
                am_span = [(i, i+len(am_tokenized)-1) for i in range(len(paragraph_tokenized)) if paragraph_tokenized[i:i+len(am_tokenized)] == am_tokenized]
                if len(am_span) > 1:
                    # print(am_span)
                    am_span = [am_span[0]]
                    # print(am_span)
                    
                    # if len(am_span) > 1:
                        
                        # print(am_span)
            
            sample_am_spans.append(am_span)
            
        # this can be done simply. see about this. 
            
        for idx_am, am_span_i in enumerate(sample_am_spans):
                
            for idx_ac, ac_span_i in enumerate(sample_ac_spans):
                
                if idx_am == idx_ac:
                    
                    adu_span = [am_span_i, ac_span_i]
                    para_spans.append(adu_span)
        
        spans.append(para_spans)
                    
        
            
    # print(len(spans))
    # print(len(ac_spans))
    
    a = []
    
    for idx, list_b in enumerate(spans):
        
        for l_l in list_b:
            
            # print(l_l)
            # y.append(len(l_l))
            
            for l_l_l in l_l:
                
                a.append(len(l_l_l))
                
                if len(l_l_l) > 1:
                    
                    print(l_l_l)
                    
    
    print(set(a))
    
    
    
    
    
    max_length = max([len(l) for l in spans])
    # print(max_length)
    
    # this is wrong. 144 is not possible.
    # ok this is correct now. 12 is correct.
    
    span_ll = []
    
    for idx, span in enumerate(spans):
        
        tmp_l = spans[idx] + (max_length - len(span)) * [[[(-1,-1)], [(-1,-1)]]]
        span_ll.append(tmp_l)
    
    # print(len(span_ll))
    
    # seto = set([len(l) for l in span_ll])
    
    # print(seto)
    
    # ok this is correct.
    
    # print(span_ll)
    
    z = []
    
    for idx, list_a in enumerate(span_ll):
        
        for l in list_a:
            
            if len(l) > 2:
                
                z.append(idx)
                
    # print(z)
    
    y = []
    x = []
    
                    
                    
    for idx, list_b in enumerate(span_ll):
        
        for l_l in list_b:
            
            # print(l_l)
            y.append(len(l_l))
            
            for l_l_l in l_l:
                
                x.append(len(l_l_l))
                    
        
    print(set(y))
    print(len(y))
    print(set(x))
    
  
    
    tokenized_text['spans'] = torch.tensor(span_ll) # then correct this.
    
    return tokenized_text

In [199]:
tokenized_input = tokenize_get_spans_align(dataset)

{1}
{2}
20628
{1}


In [200]:
# i think this is the correct one.
# make it a tensor though.

In [201]:
tokenized_input

{'input_ids': tensor([[ 101, 2009, 2003,  ...,    0,    0,    0],
        [ 101, 8821, 1010,  ...,    0,    0,    0],
        [ 101, 2034, 1997,  ...,    0,    0,    0],
        ...,
        [ 101, 1999, 7091,  ...,    0,    0,    0],
        [ 101, 2006, 1996,  ...,    0,    0,    0],
        [ 101, 2012, 1996,  ...,    0,    0,    0]]), 'token_type_ids': tensor([[0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        ...,
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        ...,
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0]]), 'labels': tensor([[ 0, -1, -1,  ..., -1, -1, -1],
        [ 0, -1, -1,  ..., -1, -1, -1],
        [ 1,  2,  2,  ..., -1, -1, -1],
        ...,
        [ 0, -1, -1,  ..., -1, -1, -1],
  

In [202]:
tokenized_input['input_ids'].shape

torch.Size([1719, 200])

In [203]:
tokenized_input['labels'].shape

torch.Size([1719, 12])

In [204]:
tokenized_input['spans'].shape

torch.Size([1719, 12, 2, 1, 2])